In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
data_path = "data/spotify.csv"
try:
    df = pd.read_csv(data_path)
except:
    # Achtung der Pfad "../../../_Daten" muss existieren.
    import kagglehub
    import shutil
    # Download latest version
    path = kagglehub.dataset_download("atharvasoundankar/spotify-global-streaming-data-2024")
    
    # Get the first file in path
    files = os.listdir(path)
    if len(files) == 0:
        raise ValueError("No files found in the downloaded dataset.")
    # Check if the data directory exists, if not create it
    if not os.path.exists("data"):
        os.makedirs("data")

    # Move the downloaded file to the desired directory
    shutil.move(os.path.join(path, files[0]), data_path)

    print("Spotifiy dataset downloaded successfully!")

In [ ]:
df.head()

In [ ]:
df.keys()

In [ ]:
# Drawing a regression line between Release Year and total streams
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

x_name = 'Release Year'
y_name = 'Total Streams (Millions)'
category_name = 'Country'

# Filter the DataFrame to include only the relevant columns
df_filtered = df[[x_name, y_name, category_name]].dropna()

# Convert the Release Year to numeric
df_filtered[x_name] = pd.to_numeric(df_filtered[x_name], errors='coerce')

# Drop rows with NaN values after conversion
df_filtered = df_filtered.dropna()

# Fitting regression model
X = df_filtered[[x_name]].values.reshape(-1, 1)
y = df_filtered[y_name].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
# Plotting the data and the regression line with confidence intervals

sns.lmplot(
    data=df_filtered, 
    x=x_name, 
    y=y_name,
    hue=category_name, 
    ci= 95,
    scatter_kws={'s':12},)
plt.title(f'{y_name} vs {x_name} with Regression Line')
plt.xlabel(x_name)
plt.ylabel(y_name)
plt.show()

# Hauptkomponenten Analyse
Nach dem festgestellt wurden das die Skip Rate durch mehrere Faktoren beeinflusst werden, soll die folgende Haupkomponenten Analyse aufklären, welche Faktoren am meisten zur Aufklärung der Varianz beitragen.

Zunächst werden die kategorialen Variablen des Datensatzes mit einem One-Hot-Encoding umgeformt.

In [ ]:
# Teilung der Test Daten zur Vorhersage der Skip Rate (%) mit Kodierung kategorialer Variablen
# Kategoriale Variablen werden automatisch per One-Hot-Encoding kodiert.
# Feature- und Zielspalten
feature_cols = [
    'Country', 
    'Artist', 
    'Album', 
    'Genre', 
    'Release Year',
    'Monthly Listeners (Millions)', 
    'Total Streams (Millions)',
    'Total Hours Streamed (Millions)', 
    'Avg Stream Duration (Min)',
    'Streams Last 30 Days (Millions)'
]

target_col = 'Skip Rate (%)'

# Filtere und bereite die Daten vor
df_reg = df[feature_cols + [target_col]].dropna()
X = df_reg[feature_cols]
y = df_reg[target_col]

# Kategoriale Variablen kodieren mit One-Hot-Encoding
X_encoded = pd.get_dummies(X, drop_first=True)

Bevor eine Hauptkomponentenanalyse durchgeführt wird, werden alle Variablen Standertisiert um Vergleichbarkeit zwischen den Variablen herzustellen.

In [ ]:
# Hauptkomponentenanalyse (PCA) auf den kodierten Features
# Die Features werden vor der PCA standardisiert.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)


Zur Selektion der Hauptkomponenten suchen wir zunächst, wie viele Komponenten es brauch um 90% der Varianz der Daten zu erklären. 
Die gefundenen Komponenten stellen wir nun mit ihrem Eigenwert in einem Scree-Plot dar. An diesem kann ein Signifikatner Sprung identifiziert werden, um zu Entscheiden ab welchen Komponentenanzahl der Cutoff gewählt wird.
Visuell auffällig sind die Sprunge zur Zweiten, Zwölften und Dreizehnten Komponente. Alle Komponente zwischen von dem Ersten bis zur 13. haben einen Eigenwert größer als eins und genügen damit dem Kaiser-Kriterium. 
Da eine weitere Analyse mit nur einem Kriterium reduntant wäre, wird sich für den nächsten signifikanten Knick an der Komponente zwölf entschieden. 

In [ ]:
# Führe PCA durch, sodass mindestens 90% der Varianz erklärt werden
pca = PCA(n_components=0.9, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

# Scree-Plot: Eigenwerte der Hauptkomponenten
plt.figure(figsize=(8,4))
plt.plot(range(1, len(pca.explained_variance_)+1), pca.explained_variance_, marker='o')
plt.axhline(1, color='red', linestyle='--', label='Grenze = 1')
plt.xlabel('Hauptkomponente')
plt.ylabel('Eigenwert')
plt.title('Scree-Plot der Hauptkomponenten (PCA)')
plt.legend()
plt.tight_layout()
plt.show()


Die zwölf selektieren Komponeten erkären gemeinsam 45,66% der Varianz der Daten 

In [ ]:
pca = PCA(n_components=12, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

print(f"Erklärte Varianz: {np.sum(pca.explained_variance_ratio_)*100:.2f}%")


Nun stellen wir die Ladungen der Variablen auf den Komponenten dar. Dazu wird eine Varimax transformation genutzt um starke Ladungen hervorzuheben und schwache Ladungen zu unterdrücken. 
An der Resultierenden Übersicht ist auffällig das jede Komponente von einem Album und ihrem jeweilgen Künstler*innen dominiert wird. 
Die allgemeinerien Variablen, wie das Erscheinungsjahr oder die Monatlichen Höhrer laden tendenziel auf alle Komponenten gleichermaßen. 
An der Darstellung ist zu beachten, dass alle Werte die unter den Wert von 0,2 fallen ausgeblendet wurden und die Tabelle lesbar zu gestalten. 

In [ ]:
# Rotierte Komponentenmatrix (Varimax-Rotation)
from factor_analyzer.rotator import Rotator

# Varimax-Rotation auf die PCA-Komponentenmatrix anwenden
rotator = Rotator(method='varimax')
rotated_components = rotator.fit_transform(pca.components_.T).T

# Rotierte Komponentenmatrix als DataFrame anzeigen
rotated_df = pd.DataFrame(rotated_components, columns=X_encoded.columns, index=[f'PC{i+1}' for i in range(pca.n_components_)]).T

# Für jede Variable: Komponente mit höchster Ladung (nach Betrag) bestimmen
max_comp = rotated_df.abs().idxmax(axis=1)
max_val = rotated_df.abs().max(axis=1)
comp_num = max_comp.str.extract(r'(\d+)').astype(int)[0]

# Sortierindex erstellen: erst nach Komponente, dann nach Höhe der Ladung (absteigend)
sort_idx = pd.DataFrame({'comp': comp_num, 'val': max_val})
sort_idx = sort_idx.sort_values(['comp', 'val'], ascending=[True, False]).index
rotated_df_sorted = rotated_df.loc[sort_idx]

# Werte < 0.08 ausblenden
rotated_df_masked = rotated_df_sorted.map(lambda x: x if abs(x) >= 0.2 else "")

# Trick um das gesamte DataFrame anzuzeigen
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(rotated_df_masked)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

# Entscheidungsbaum
Als Alternative Methode zur multiplen linearen Regression, welche die Skip Rate beeinflusst, wird ein Regressionsentscheidungsbaum trainiert.

Hierzu teilen wir den Datensatz in 20% Testdaten und 80% Trainingsdaten.

In [ ]:
# Splitte die Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

Da die optimalen Parameter für das Training eines Entscheidungsbaums mit den gegeben Daten unbekannt ist, wird mit einer Gridsuche systematisch Parameter ausprobiert. Es werden jene Parameter ausgewählt, welche in dem größten Wert des Bestimmtheitsmaßes, R², resultieren.

In [ ]:
# Grid Search zur Optimierung der Entscheidungsbaum-Parameter
param_grid = {
    'max_depth': [4, 6, 8, 12, None],
    'min_impurity_decrease': [0.0, 0.01, 0.1, 0.5],
    'min_samples_split': [2, 5, 10, 20]
}

grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print("Beste Parameterkombination:")
print(grid_search.best_params_)
print(f"Bester mittlerer R²-Score: {grid_search.best_score_:.3f}")

# Optional: Ergebnisse als DataFrame anzeigen
# results_df = pd.DataFrame(grid_search.cv_results_)
# display(results_df[['params', 'mean_test_score', 'std_test_score']].sort_values('mean_test_score', ascending=False).head(10))

Mit den nun bestimmten Parametern wird ein Modell trainiert, welches für weitere Berechnungen verwendet wird.

In [ ]:

# Trainiere den Regressionsbaum
reg_tree = DecisionTreeRegressor(**grid_search.best_params_, random_state=42)
reg_tree.fit(X_train, y_train)

Um ein Overfitting des Modells zu kontrollieren wird eine Kreuzvalidierung über 5 Holdouts berechnet. Das Ergebnis weißt eine akzeptierbare Streuung auf in an betracht der geringen Datenmenge.

In [ ]:
# Cross-Validierung für den Regressionsentscheidungsbaum
splits = 5
cv = KFold(n_splits=splits, shuffle=True, random_state=42)

# R²-Scores
r2_scores = cross_val_score(reg_tree, X_encoded, y, cv=cv, scoring='r2')
print(f"Mittlerer R²-Score ({splits}-fold CV): {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

# MSE-Scores (negativ, daher Vorzeichen umdrehen)
mse_scores = cross_val_score(reg_tree, X_encoded, y, cv=cv, scoring=make_scorer(mean_squared_error, greater_is_better=False))
print(f"Mittlerer MSE ({splits}-fold CV): {(-np.mean(mse_scores)):.3f} ± {np.std(mse_scores):.3f}")

Final wird ein Regressionsbaum trainiert dessen Bestimmtheitsmaß negativ ist. Dies lässt darauf schließen, dass die Model schlechter auf die Daten passt als eine vaagerechte Line. Abschließend ist daher zu folgern, dass eine Vorhersage der Skip Rate mittels eines Regressionentscheidungsbaum nicht möglich ist. 

In [ ]:
# Vorhersage und Bewertung
y_pred = reg_tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.3f}, R²: {r2:.3f}")


# Anhang
Im Folgeden sind die Darstellungen der Wichtigkeiten der Variablen innerhalb des Entscheidungsbaums dargestellt. Zu erst werden die Wichtigkeit aller Variablen dargestellt. Darauf Folgenden werden Wichtigkiet der kategorialen Variablen gruppiert dargestellt. 

In [ ]:
# Feature-Importances anzeigen
importances = reg_tree.feature_importances_
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(12,5))
plt.title("Feature Importances (Top 15)")
# Zeige nur die wichtigsten 15 Features an
top_n = 15
plt.bar(range(top_n), importances[indices][:top_n], align="center")
plt.xticks(range(top_n), [X_encoded.columns[i] for i in indices[:top_n]], rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict

# Feature-Importances gruppiert nach Ursprungsvariablen anzeigen
importances = reg_tree.feature_importances_
feature_names = X_encoded.columns

# Ursprungsvariablen bestimmen
orig_vars = feature_cols

# Initialisiert ein Dictionary mit Listen für jede Ursprungsvariable
var_to_cols = defaultdict(list)
for col in feature_names:
    found = False
    for var in orig_vars:
        if col == var or col.startswith(var + '_'):
            var_to_cols[var].append(col)
            found = True
            break
    if not found:
        var_to_cols[col].append(col)  # falls neue Spalten auftauchen

# Importance pro Ursprungsvariable aufsummieren
summed_importances = {var: importances[[feature_names.get_loc(c) for c in cols]].sum() for var, cols in var_to_cols.items()}
# Sortieren
sorted_vars = sorted(summed_importances, key=summed_importances.get, reverse=True)

plt.figure(figsize=(10,5))
plt.title("Feature Importances (gruppiert nach Ursprungsvariablen)")
plt.bar(range(len(sorted_vars)), [summed_importances[v] for v in sorted_vars], align="center")
plt.xticks(range(len(sorted_vars)), sorted_vars, rotation=45, ha='right')
plt.tight_layout()
plt.show()